In [8]:
import requests
import json
openrouter_key='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'
response = requests.post(
  url="https://openrouter.ai/api/v1/chat/completions",
  headers={
    "Authorization": f"Bearer {openrouter_key}",
    "Content-Type": "application/json",
  },
  data=json.dumps({
    "model": "deepseek/deepseek-chat-v3.1:free",
    "messages": [
      {
        "role": "user",
        "content": "解释一下多因子策略在投资组合中的作用?用中文回答"
      }
    ],
    
  })
)

In [7]:
response.json()["choices"][0]["message"]["content"]

KeyError: 'choices'

In [5]:
import requests
import json
openrouter_key='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'
response = requests.get(
  url="https://openrouter.ai/api/v1/key",
  headers={
    "Authorization": f"Bearer {openrouter_key}"
  }
)
print(json.dumps(response.json(), indent=2))

{
  "error": {
    "message": "User not found.",
    "code": 401
  }
}


In [4]:
import pandas as pd 
df=pd.read_parquet(r'D:\projects\q\data\minute_data\000002\2025-09.parquet')

In [5]:
df

,date,open,high,low,close,volume
0,2025-09-03 13:53:00,6.60,6.60,6.59,6.60,224300
1,2025-09-03 13:54:00,6.60,6.60,6.59,6.60,695600
2,2025-09-03 13:55:00,6.60,6.61,6.60,6.61,339600
3,2025-09-03 13:56:00,6.61,6.61,6.60,6.61,406800
4,2025-09-03 13:57:00,6.61,6.61,6.59,6.60,446900
...,...,...,...,...,...,...
1965,2025-09-15 14:54:00,NaN,NaN,NaN,NaN,649200
1966,2025-09-15 14:55:00,NaN,NaN,NaN,NaN,1186100
1967,2025-09-15 14:56:00,NaN,NaN,NaN,NaN,2255700
1968,2025-09-15 14:57:00,NaN,NaN,NaN,NaN,2261800


In [16]:
import requests
import json
import pandas as pd
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import re

class EastmoneyNewsCrawler:
    def __init__(self):
        self.api_url = "https://search-api-web.eastmoney.com/search/jsonp"
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Referer': 'https://so.eastmoney.com/',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        }
    
    def get_news_content(self, keyword, start_date=None, end_date=None, max_pages=2):
        """
        获取新闻内容 - 简化版，先测试基本功能
        """
        
        print(f"搜索关键词: {keyword}")
        print(f"获取页数: {max_pages}")
        
        all_news = []
        
        for page in range(1, max_pages + 1):
            # 获取新闻列表
            news_list = self._get_news_list(keyword, page)
            if not news_list:
                break
                
            # 获取每页前2条新闻的内容进行测试
            for i, news in enumerate(news_list[:2]):
                news_date = self._parse_date(news.get('raw_date', ''))
                
                print(f"\n正在处理第{i+1}条新闻:")
                print(f"  标题: {news['title'][:60]}...")
                print(f"  日期: {news_date}")
                print(f"  来源: {news['source']}")
                print(f"  URL: {news['url']}")
                
                content = self._fetch_news_content(news['url'])
                if content:
                    all_news.append({
                        'title': news['title'].replace('<em>', '').replace('</em>', ''),  # 清理高亮标签
                        'date': news_date,
                        'url': news['url'],
                        'source': news['source'],
                        'content': content
                    })
                    print(f"  ✓ 内容获取成功 ({len(content)} 字符)")
                else:
                    print(f"  ✗ 内容获取失败")
                    
                time.sleep(1)
            
            time.sleep(2)
        
        # 转换为DataFrame并按时间排序
        if all_news:
            df = pd.DataFrame(all_news)
            try:
                df['date'] = pd.to_datetime(df['date'])
                df = df.sort_values('date', ascending=False).reset_index(drop=True)
            except:
                pass  # 如果日期解析失败，不排序
            
            print(f"\n总共获取 {len(df)} 条新闻内容")
            return df
        else:
            print("未获取到任何新闻内容")
            return pd.DataFrame()
    
    def _get_news_list(self, keyword, page_index):
        """获取新闻列表"""
        param_data = {
            "uid": "",
            "keyword": keyword,
            "type": ["article"],
            "client": "web",
            "clientType": "web",
            "clientVersion": "curr",
            "param": {
                "article": {
                    "searchScope": "",
                    "sort": "time",  # 修改为按时间排序
                    "pageIndex": page_index,
                    "pageSize": 20,
                    "preTag": "",
                    "postTag": ""
                }
            }
        }
        
        params = {
            'cb': f'jQuery{int(time.time() * 1000)}_{int(time.time() * 1000) + 1}',
            'param': json.dumps(param_data)
        }
        
        try:
            response = requests.get(self.api_url, headers=self.headers, params=params, timeout=15)
            if response.status_code == 200:
                response_text = response.text
                
                if response_text.startswith('jQuery'):
                    start = response_text.find('(') + 1
                    end = response_text.rfind(')')
                    json_str = response_text[start:end]
                else:
                    json_str = response_text
                
                data = json.loads(json_str)
                
                if 'result' in data and data['result'] and 'article' in data['result']:
                    articles = data['result']['article']
                    print(f"第{page_index}页获取到 {len(articles)} 条新闻")
                    
                    # 调试第一条新闻的字段
                    if articles:
                        first_article = articles[0]
                        print(f"新闻字段: {list(first_article.keys())}")
                        print(f"date: {first_article.get('date')}")
                        print(f"示例标题: {first_article.get('title')[:50]}...")
                    
                    news_list = []
                    for article in articles:
                        # 使用正确的日期字段
                        article_date = article.get('date', '')
                        formatted_date = self._parse_date(article_date)
                        
                        news_list.append({
                            'title': article.get('title', '').strip(),
                            'url': article.get('url', ''),
                            'date': formatted_date,
                            'source': article.get('nickname', ''),  # 使用nickname作为来源
                            'raw_date': article_date  # 保留原始日期用于调试
                        })
                    
                    return news_list
                    
        except Exception as e:
            print(f"获取第{page_index}页失败: {e}")
        
        return []
    
    def _fetch_news_content(self, url):
        """获取新闻正文内容"""
        if not url:
            return ""
            
        try:
            response = requests.get(url, headers=self.headers, timeout=10)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                
                # 尝试多种可能的正文选择器
                content_selectors = [
                    '.ArticleBody',
                    '.content',
                    '.article-content',
                    '#ContentBody',
                    '.main-content',
                    'article',
                    '.post-content'
                ]
                
                for selector in content_selectors:
                    content_div = soup.select_one(selector)
                    if content_div:
                        text = content_div.get_text().strip()
                        text = re.sub(r'\s+', ' ', text)
                        if len(text) > 100:
                            return text
                
                # 如果没找到，尝试获取所有p标签内容
                paragraphs = soup.find_all('p')
                if paragraphs:
                    content = ' '.join([p.get_text().strip() for p in paragraphs])
                    content = re.sub(r'\s+', ' ', content)
                    if len(content) > 100:
                        return content
                        
        except Exception as e:
            print(f"获取内容失败 {url}: {e}")
        
        return ""
    
    def _parse_date(self, date_str):
        """解析日期字符串"""
        if not date_str:
            return ""
        
        try:
            # 处理ISO格式
            if 'T' in str(date_str):
                dt = datetime.fromisoformat(str(date_str).replace('Z', ''))
                return dt.strftime('%Y-%m-%d %H:%M:%S')
            
            # 处理其他常见格式
            date_str = str(date_str).strip()
            
            if re.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', date_str):
                return date_str
            
            if re.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}', date_str):
                return date_str + ':00'
            
            if re.match(r'\d{4}-\d{2}-\d{2}', date_str):
                return date_str + ' 00:00:00'
                
        except Exception:
            pass
        
        return date_str
    
    def _is_in_date_range(self, news_date, start_date, end_date):
        """检查日期是否在指定范围内"""
        try:
            if not news_date:
                return True
                
            news_dt = datetime.strptime(news_date[:10], '%Y-%m-%d')
            start_dt = datetime.strptime(start_date, '%Y-%m-%d')
            end_dt = datetime.strptime(end_date, '%Y-%m-%d')
            
            return start_dt <= news_dt <= end_dt
        except Exception:
            return True

# 使用示例
if __name__ == "__main__":
    crawler = EastmoneyNewsCrawler()
    
    # 获取紫金矿业最近一周的新闻内容
    df = crawler.get_news_content("紫金矿业", max_pages=2)
    
    if not df.empty:
        print(f"\n获取到 {len(df)} 条新闻")
        print("\n最新3条新闻:")
        for i, row in df.head(3).iterrows():
            print(f"\n{i+1}. {row['title']}")
            print(f"时间: {row['date']}")
            print(f"来源: {row['source']}")
            print(f"内容: {row['content'][:200]}...")
        
        # 保存到CSV
        df.to_csv('news_content.csv', index=False, encoding='utf-8-sig')
        print(f"\n数据已保存到 news_content.csv")
    else:
        print("未获取到新闻数据")

搜索关键词: 紫金矿业
获取页数: 2
第1页获取到 20 条新闻
新闻字段: ['id', 'code', 'title', 'postId', 'nickname', 'content', 'date', 'articleType', 'portrait', 'listImage', 'url', 'authorUrl', 'uid', 'likeNum', 'commentNum']
date: 2023-12-28 08:30:00
示例标题: 后浪森林·商业史 | <em>紫金矿业</em>三十二年，人人都是大富翁（四）...

正在处理第1条新闻:
  标题: 后浪森林·商业史 | <em>紫金矿业</em>三十二年，人人都是大富翁（四）...
  日期: 2023-12-28 08:30:00
  来源: 成长企业常识
  URL: http://caifuhao.eastmoney.com/news/20231228004008763123340
  ✗ 内容获取失败

正在处理第2条新闻:
  标题: <em>紫金矿业</em>，爆了！...
  日期: 2025-02-13 16:28:05
  来源: 看财经
  URL: http://caifuhao.eastmoney.com/news/20250213162805586513200
  ✗ 内容获取失败
第2页获取到 20 条新闻
新闻字段: ['id', 'code', 'title', 'postId', 'nickname', 'content', 'date', 'articleType', 'portrait', 'listImage', 'url', 'authorUrl', 'uid', 'likeNum', 'commentNum']
date: 2024-11-14 19:39:05
示例标题: <em>紫金矿业</em>遭遇“黄金大劫案”？公司回应...

正在处理第1条新闻:
  标题: <em>紫金矿业</em>遭遇“黄金大劫案”？公司回应...
  日期: 2024-11-14 19:39:05
  来源: 财经读数
  URL: http://caifuhao.eastmoney.com/news/20241114193905492120760
  ✗ 内容获

搜索关键词: 紫金矿业
获取页数: 2
第1页获取到 20 条新闻
新闻字段: ['id', 'code', 'title', 'postId', 'nickname', 'content', 'date', 'articleType', 'portrait', 'listImage', 'url', 'authorUrl', 'uid', 'likeNum', 'commentNum']
date: 2023-12-28 08:30:00
示例标题: 后浪森林·商业史 | <em>紫金矿业</em>三十二年，人人都是大富翁（四）...

正在处理第1条新闻:
  标题: 后浪森林·商业史 | <em>紫金矿业</em>三十二年，人人都是大富翁（四）...
  日期: 2023-12-28 08:30:00
  来源: 成长企业常识
  URL: http://caifuhao.eastmoney.com/news/20231228004008763123340
  ✓ 内容获取成功 (4089 字符)

正在处理第2条新闻:
  标题: <em>紫金矿业</em>，爆了！...
  日期: 2025-02-13 16:28:05
  来源: 看财经
  URL: http://caifuhao.eastmoney.com/news/20250213162805586513200
  ✓ 内容获取成功 (1527 字符)
第2页获取到 20 条新闻
新闻字段: ['id', 'code', 'title', 'postId', 'nickname', 'content', 'date', 'articleType', 'portrait', 'listImage', 'url', 'authorUrl', 'uid', 'likeNum', 'commentNum']
date: 2024-11-14 19:39:05
示例标题: <em>紫金矿业</em>遭遇“黄金大劫案”？公司回应...

正在处理第1条新闻:
  标题: <em>紫金矿业</em>遭遇“黄金大劫案”？公司回应...
  日期: 2024-11-14 19:39:05
  来源: 财经读数
  URL: http://caifuhao.eastmoney.com/news/20241114193

In [21]:
df

,title,date,url,source,content
0,紫金矿业遭遇“黄金大劫案”？公司回应,2024-11-14 19:39:05,http://caifuhao.eastmoney.com/news/20241114193...,财经读数,中国企业投资的海外矿山存在被盗采的情况，整体来看不多见，但在一些高风险国家，类似事件比较多 ...
1,热衷买矿的紫金矿业，又大幅融资补血了,2024-06-19 23:41:01,http://caifuhao.eastmoney.com/news/20240619234...,雷达财经,雷达财经鸿途出品 文|李亦辉 编|深海 有着“矿茅”之称的紫金矿业，正在大举融资。 6月18...
2,后浪森林·商业史 | 紫金矿业三十二年，人人都是大富翁（四）,2023-12-28 08:30:00,http://caifuhao.eastmoney.com/news/20231228004...,成长企业常识,后浪森林研究室|三文 编辑|罗周 统筹、编辑助理|许佳维 紫金矿业改变了太多人的命运。 从陈...
3,刘肥肥：紫金矿业4月17日午评,2023-04-17 15:54:02,http://caifuhao.eastmoney.com/news/20230417155...,刘肥肥,1，紫金矿业 601899 今日收盘价13.91元人民币， 港股紫金矿业 02899下午三点...


In [23]:
import requests
import json
import pandas as pd
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import re

class EastmoneyNewsCrawler:
    def __init__(self):
        self.api_url = "https://search-api-web.eastmoney.com/search/jsonp"
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Referer': 'https://so.eastmoney.com/',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        }
    
    def get_news_content(self, keyword, start_date=None, end_date=None, max_pages=2):
        """
        获取新闻内容 - 简化版，先测试基本功能
        """
        
        print(f"搜索关键词: {keyword}")
        print(f"获取页数: {max_pages}")
        
        all_news = []
        
        for page in range(1, max_pages + 1):
            # 获取新闻列表
            news_list = self._get_news_list(keyword, page)
            if not news_list:
                break
                
            # 获取每页前2条新闻的内容进行测试
            for i, news in enumerate(news_list[:2]):
                news_date = self._parse_date(news.get('raw_date', ''))
                
                print(f"\n正在处理第{i+1}条新闻:")
                print(f"  标题: {news['title'][:60]}...")
                print(f"  日期: {news_date}")
                print(f"  来源: {news['source']}")
                print(f"  URL: {news['url']}")
                
                content = self._fetch_news_content(news['url'])
                if content:
                    all_news.append({
                        'title': news['title'].replace('<em>', '').replace('</em>', ''),  # 清理高亮标签
                        'date': news_date,
                        'url': news['url'],
                        'source': news['source'],
                        'content': content
                    })
                    print(f"  ✓ 内容获取成功 ({len(content)} 字符)")
                else:
                    print(f"  ✗ 内容获取失败")
                    
                time.sleep(1)
            
            time.sleep(2)
        
        # 转换为DataFrame并按时间排序
        if all_news:
            df = pd.DataFrame(all_news)
            try:
                df['date'] = pd.to_datetime(df['date'])
                df = df.sort_values('date', ascending=False).reset_index(drop=True)
            except:
                pass  # 如果日期解析失败，不排序
            
            print(f"\n总共获取 {len(df)} 条新闻内容")
            return df
        else:
            print("未获取到任何新闻内容")
            return pd.DataFrame()

    def test_sorting(self, keyword):
        """测试不同排序参数的效果"""
        
        sort_values = ["default", "relate", "time", "", "1", "2"]
        
        for sort_val in sort_values:
            print(f"\n=== 测试排序参数: '{sort_val}' ===")
            
            param_data = {
                "uid": "",
                "keyword": keyword,
                "type": ["article"],
                "client": "web",
                "clientType": "web",
                "clientVersion": "curr",
                "param": {
                    "article": {
                        "searchScope": "",
                        "sort": sort_val,
                        "pageIndex": 1,
                        "pageSize": 5,
                        "preTag": "",
                        "postTag": ""
                    }
                }
            }
            
            params = {
                'cb': f'jQuery{int(time.time() * 1000)}_{int(time.time() * 1000) + 1}',
                'param': json.dumps(param_data)
            }
            
            try:
                response = requests.get(self.api_url, headers=self.headers, params=params, timeout=10)
                if response.status_code == 200:
                    response_text = response.text
                    
                    if response_text.startswith('jQuery'):
                        start = response_text.find('(') + 1
                        end = response_text.rfind(')')
                        json_str = response_text[start:end]
                    else:
                        json_str = response_text
                    
                    data = json.loads(json_str)
                    
                    if 'result' in data and data['result'] and 'article' in data['result']:
                        articles = data['result']['article']
                        print(f"获取到 {len(articles)} 条新闻")
                        
                        # 显示前5条新闻的日期，看排序效果
                        dates = []
                        for i, article in enumerate(articles):
                            date = article.get('date', '')
                            title = article.get('title', '').replace('<em>', '').replace('</em>', '')[:40]
                            print(f"  {i+1}. {date} - {title}...")
                            dates.append(date)
                        
                        # 检查是否按时间排序（最新在前）
                        if len(dates) > 1:
                            is_desc_sorted = all(dates[i] >= dates[i+1] for i in range(len(dates)-1) if dates[i] and dates[i+1])
                            print(f"是否按时间倒序排列: {is_desc_sorted}")
                    else:
                        print("没有获取到新闻")
                        
            except Exception as e:
                print(f"测试失败: {e}")
            
            time.sleep(1)
        """测试不同的排序参数"""
        
        sort_options = [
            "",              # 空字符串（默认）
            "default",       # 默认
            "time",          # 时间
            "relate",        # 相关度
            "score",         # 分数
            "1",             # 数字值
            "2", 
            "3",
            "createTime",    # 创建时间
            "publishTime",   # 发布时间
            "desc",          # 降序
            "asc",           # 升序
        ]
        
        for sort_param in sort_options:
            print(f"\n=== 测试排序参数: '{sort_param}' ===")
            
            param_data = {
                "uid": "",
                "keyword": keyword,
                "type": ["article"],
                "client": "web",
                "clientType": "web",
                "clientVersion": "curr",
                "param": {
                    "article": {
                        "searchScope": "",
                        "sort": sort_param,
                        "pageIndex": 1,
                        "pageSize": 5,
                        "preTag": "",
                        "postTag": ""
                    }
                }
            }
            
            params = {
                'cb': f'jQuery{int(time.time() * 1000)}_{int(time.time() * 1000) + 1}',
                'param': json.dumps(param_data)
            }
            
            try:
                response = requests.get(self.api_url, headers=self.headers, params=params, timeout=10)
                if response.status_code == 200:
                    response_text = response.text
                    
                    if response_text.startswith('jQuery'):
                        start = response_text.find('(') + 1
                        end = response_text.rfind(')')
                        json_str = response_text[start:end]
                    else:
                        json_str = response_text
                    
                    data = json.loads(json_str)
                    
                    if 'result' in data and data['result'] and 'article' in data['result']:
                        articles = data['result']['article']
                        print(f"获取到 {len(articles)} 条新闻")
                        
                        # 显示前3条新闻的日期
                        for i, article in enumerate(articles[:3]):
                            date = article.get('date', '')
                            title = article.get('title', '')[:50]
                            print(f"  {i+1}. {date} - {title}...")
                    else:
                        print("没有获取到新闻")
                        
            except Exception as e:
                print(f"测试失败: {e}")
            
            time.sleep(1)
        """
        获取新闻内容 - 简化版，先测试基本功能
        """
        
        print(f"搜索关键词: {keyword}")
        print(f"获取页数: {max_pages}")
        
        all_news = []
        
        for page in range(1, max_pages + 1):
            # 获取新闻列表
            news_list = self._get_news_list(keyword, page)
            if not news_list:
                break
                
            # 获取每页前2条新闻的内容进行测试
            for i, news in enumerate(news_list[:2]):
                news_date = self._parse_date(news.get('raw_date', ''))
                
                print(f"\n正在处理第{i+1}条新闻:")
                print(f"  标题: {news['title'][:60]}...")
                print(f"  日期: {news_date}")
                print(f"  来源: {news['source']}")
                print(f"  URL: {news['url']}")
                
                content = self._fetch_news_content(news['url'])
                if content:
                    all_news.append({
                        'title': news['title'].replace('<em>', '').replace('</em>', ''),  # 清理高亮标签
                        'date': news_date,
                        'url': news['url'],
                        'source': news['source'],
                        'content': content
                    })
                    print(f"  ✓ 内容获取成功 ({len(content)} 字符)")
                else:
                    print(f"  ✗ 内容获取失败")
                    
                time.sleep(1)
            
            time.sleep(2)
        
        # 转换为DataFrame并按时间排序
        if all_news:
            df = pd.DataFrame(all_news)
            try:
                df['date'] = pd.to_datetime(df['date'])
                df = df.sort_values('date', ascending=False).reset_index(drop=True)
            except:
                pass  # 如果日期解析失败，不排序
            
            print(f"\n总共获取 {len(df)} 条新闻内容")
            return df
        else:
            print("未获取到任何新闻内容")
            return pd.DataFrame()
    
    def _get_news_list(self, keyword, page_index):
        """获取新闻列表"""
        param_data = {
            "uid": "",
            "keyword": keyword,
            "type": ["article"],
            "client": "web",
            "clientType": "web",
            "clientVersion": "curr",
            "param": {
                "article": {
                    "searchScope": "",
                    "sort": "time",  # 使用正确的时间排序参数
                    "pageIndex": page_index,
                    "pageSize": 20,
                    "preTag": "",
                    "postTag": ""
                }
            }
        }
        
        params = {
            'cb': f'jQuery{int(time.time() * 1000)}_{int(time.time() * 1000) + 1}',
            'param': json.dumps(param_data)
        }
        
        try:
            response = requests.get(self.api_url, headers=self.headers, params=params, timeout=15)
            if response.status_code == 200:
                response_text = response.text
                
                if response_text.startswith('jQuery'):
                    start = response_text.find('(') + 1
                    end = response_text.rfind(')')
                    json_str = response_text[start:end]
                else:
                    json_str = response_text
                
                data = json.loads(json_str)
                
                if 'result' in data and data['result'] and 'article' in data['result']:
                    articles = data['result']['article']
                    print(f"第{page_index}页获取到 {len(articles)} 条新闻")
                    
                    # 调试第一条新闻的字段
                    if articles:
                        first_article = articles[0]
                        print(f"新闻字段: {list(first_article.keys())}")
                        print(f"date: {first_article.get('date')}")
                        print(f"示例标题: {first_article.get('title')[:50]}...")
                    
                    news_list = []
                    for article in articles:
                        # 使用正确的日期字段
                        article_date = article.get('date', '')
                        formatted_date = self._parse_date(article_date)
                        
                        news_list.append({
                            'title': article.get('title', '').strip(),
                            'url': article.get('url', ''),
                            'date': formatted_date,
                            'source': article.get('nickname', ''),  # 使用nickname作为来源
                            'raw_date': article_date  # 保留原始日期用于调试
                        })
                    
                    return news_list
                    
        except Exception as e:
            print(f"获取第{page_index}页失败: {e}")
        
        return []
    
    def _fetch_news_content(self, url):
        """获取新闻正文内容"""
        if not url:
            return ""
            
        try:
            response = requests.get(url, headers=self.headers, timeout=10)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                
                # 尝试多种可能的正文选择器
                content_selectors = [
                    '.ArticleBody',
                    '.content',
                    '.article-content',
                    '#ContentBody',
                    '.main-content',
                    'article',
                    '.post-content'
                ]
                
                for selector in content_selectors:
                    content_div = soup.select_one(selector)
                    if content_div:
                        text = content_div.get_text().strip()
                        text = re.sub(r'\s+', ' ', text)
                        if len(text) > 100:
                            return text
                
                # 如果没找到，尝试获取所有p标签内容
                paragraphs = soup.find_all('p')
                if paragraphs:
                    content = ' '.join([p.get_text().strip() for p in paragraphs])
                    content = re.sub(r'\s+', ' ', content)
                    if len(content) > 100:
                        return content
                        
        except Exception as e:
            print(f"获取内容失败 {url}: {e}")
        
        return ""
    
    def _parse_date(self, date_str):
        """解析日期字符串"""
        if not date_str:
            return ""
        
        try:
            # 处理ISO格式
            if 'T' in str(date_str):
                dt = datetime.fromisoformat(str(date_str).replace('Z', ''))
                return dt.strftime('%Y-%m-%d %H:%M:%S')
            
            # 处理其他常见格式
            date_str = str(date_str).strip()
            
            if re.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', date_str):
                return date_str
            
            if re.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}', date_str):
                return date_str + ':00'
            
            if re.match(r'\d{4}-\d{2}-\d{2}', date_str):
                return date_str + ' 00:00:00'
                
        except Exception:
            pass
        
        return date_str
    
    def _is_in_date_range(self, news_date, start_date, end_date):
        """检查日期是否在指定范围内"""
        try:
            if not news_date:
                return True
                
            news_dt = datetime.strptime(news_date[:10], '%Y-%m-%d')
            start_dt = datetime.strptime(start_date, '%Y-%m-%d')
            end_dt = datetime.strptime(end_date, '%Y-%m-%d')
            
            return start_dt <= news_dt <= end_dt
        except Exception:
            return True

# 使用示例
if __name__ == "__main__":
    crawler = EastmoneyNewsCrawler()
    
    # 获取紫金矿业最近一周的新闻内容
    df = crawler.get_news_content("紫金矿业", max_pages=2)
    
    if not df.empty:
        print(f"\n获取到 {len(df)} 条新闻")
        print("\n最新3条新闻:")
        for i, row in df.head(3).iterrows():
            print(f"\n{i+1}. {row['title']}")
            print(f"时间: {row['date']}")
            print(f"来源: {row['source']}")
            print(f"内容: {row['content'][:200]}...")
        
        # 保存到CSV
        df.to_csv('news_content.csv', index=False, encoding='utf-8-sig')
        print(f"\n数据已保存到 news_content.csv")
    else:
        print("未获取到新闻数据")

搜索关键词: 紫金矿业
获取页数: 2
第1页获取到 20 条新闻
新闻字段: ['id', 'code', 'title', 'postId', 'nickname', 'content', 'date', 'articleType', 'portrait', 'listImage', 'url', 'authorUrl', 'uid', 'likeNum', 'commentNum']
date: 2023-12-28 08:30:00
示例标题: 后浪森林·商业史 | <em>紫金矿业</em>三十二年，人人都是大富翁（四）...

正在处理第1条新闻:
  标题: 后浪森林·商业史 | <em>紫金矿业</em>三十二年，人人都是大富翁（四）...
  日期: 2023-12-28 08:30:00
  来源: 成长企业常识
  URL: http://caifuhao.eastmoney.com/news/20231228004008763123340
  ✓ 内容获取成功 (4089 字符)

正在处理第2条新闻:
  标题: 热衷买矿的<em>紫金矿业</em>，又大幅融资补血了...
  日期: 2024-06-19 23:41:01
  来源: 雷达财经
  URL: http://caifuhao.eastmoney.com/news/20240619234101211306570
  ✓ 内容获取成功 (3841 字符)
第2页获取到 20 条新闻
新闻字段: ['id', 'code', 'title', 'postId', 'nickname', 'content', 'date', 'articleType', 'portrait', 'listImage', 'url', 'authorUrl', 'uid', 'likeNum', 'commentNum']
date: 2024-11-14 19:39:05
示例标题: <em>紫金矿业</em>遭遇“黄金大劫案”？公司回应...

正在处理第1条新闻:
  标题: <em>紫金矿业</em>遭遇“黄金大劫案”？公司回应...
  日期: 2024-11-14 19:39:05
  来源: 财经读数
  URL: http://caifuhao.eastmoney.com/news/

In [25]:
import requests
import json
import pandas as pd
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import re

class EastmoneyNewsCrawler:
    def __init__(self):
        self.api_url = "https://search-api-web.eastmoney.com/search/jsonp"
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Referer': 'https://so.eastmoney.com/',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        }
    
    def get_news_content(self, keyword, max_pages=2, max_news=10):
        """
        获取新闻内容（简化版）
        
        Args:
            keyword: 搜索关键词
            max_pages: 最大页数
            max_news: 最大新闻数量
            
        Returns:
            DataFrame: 包含新闻标题、时间、内容的数据框，按时间排序
        """
        
        print(f"搜索关键词: {keyword}")
        print(f"最大获取 {max_news} 条新闻")
        
        all_news = []
        
        for page in range(1, max_pages + 1):
            if len(all_news) >= max_news:
                break
                
            # 获取新闻列表
            news_list = self._get_news_list(keyword, page)
            if not news_list:
                break
                
            # 处理新闻
            for news in news_list:
                if len(all_news) >= max_news:
                    break
                    
                news_date = self._parse_date(news.get('raw_date', ''))
                
                print(f"\n正在处理: {news['title'][:50]}...")
                print(f"  日期: {news_date}")
                
                content = self._fetch_news_content(news['url'])
                if content and len(content) > 100:  # 确保内容有意义
                    all_news.append({
                        'title': news['title'].replace('<em>', '').replace('</em>', ''),
                        'date': news_date,
                        'url': news['url'],
                        'source': news['source'],
                        'content': content
                    })
                    print(f"  ✓ 成功获取内容 ({len(content)} 字符)")
                else:
                    print(f"  ✗ 内容获取失败或内容太短")
                    
                time.sleep(1)
            
            time.sleep(2)
        
        # 转换为DataFrame并手动按时间排序
        if all_news:
            df = pd.DataFrame(all_news)
            
            # 手动按时间排序（最新在前）
            try:
                df['date_parsed'] = pd.to_datetime(df['date'], errors='coerce')
                df = df.sort_values('date_parsed', ascending=False, na_last=True)
                df = df.drop('date_parsed', axis=1).reset_index(drop=True)
                print(f"\n成功获取 {len(df)} 条新闻，已按时间排序")
            except Exception as e:
                print(f"时间排序失败: {e}")
            
            return df
        else:
            print("未获取到任何新闻内容")
            return pd.DataFrame()
        """
        获取新闻内容 - 简化版，先测试基本功能
        """
        
        print(f"搜索关键词: {keyword}")
        print(f"获取页数: {max_pages}")
        
        all_news = []
        
        for page in range(1, max_pages + 1):
            # 获取新闻列表
            news_list = self._get_news_list(keyword, page)
            if not news_list:
                break
                
            # 获取每页前2条新闻的内容进行测试
            for i, news in enumerate(news_list[:2]):
                news_date = self._parse_date(news.get('raw_date', ''))
                
                print(f"\n正在处理第{i+1}条新闻:")
                print(f"  标题: {news['title'][:60]}...")
                print(f"  日期: {news_date}")
                print(f"  来源: {news['source']}")
                print(f"  URL: {news['url']}")
                
                content = self._fetch_news_content(news['url'])
                if content:
                    all_news.append({
                        'title': news['title'].replace('<em>', '').replace('</em>', ''),  # 清理高亮标签
                        'date': news_date,
                        'url': news['url'],
                        'source': news['source'],
                        'content': content
                    })
                    print(f"  ✓ 内容获取成功 ({len(content)} 字符)")
                else:
                    print(f"  ✗ 内容获取失败")
                    
                time.sleep(1)
            
            time.sleep(2)
        
        # 转换为DataFrame并按时间排序
        if all_news:
            df = pd.DataFrame(all_news)
            try:
                df['date'] = pd.to_datetime(df['date'])
                df = df.sort_values('date', ascending=False).reset_index(drop=True)
            except:
                pass  # 如果日期解析失败，不排序
            
            print(f"\n总共获取 {len(df)} 条新闻内容")
            return df
        else:
            print("未获取到任何新闻内容")
            return pd.DataFrame()

    def test_sorting(self, keyword):
        """测试不同排序参数的效果"""
        
        sort_values = ["default", "relate", "time", "", "1", "2"]
        
        for sort_val in sort_values:
            print(f"\n=== 测试排序参数: '{sort_val}' ===")
            
            param_data = {
                "uid": "",
                "keyword": keyword,
                "type": ["article"],
                "client": "web",
                "clientType": "web",
                "clientVersion": "curr",
                "param": {
                    "article": {
                        "searchScope": "",
                        "sort": sort_val,
                        "pageIndex": 1,
                        "pageSize": 5,
                        "preTag": "",
                        "postTag": ""
                    }
                }
            }
            
            params = {
                'cb': f'jQuery{int(time.time() * 1000)}_{int(time.time() * 1000) + 1}',
                'param': json.dumps(param_data)
            }
            
            try:
                response = requests.get(self.api_url, headers=self.headers, params=params, timeout=10)
                if response.status_code == 200:
                    response_text = response.text
                    
                    if response_text.startswith('jQuery'):
                        start = response_text.find('(') + 1
                        end = response_text.rfind(')')
                        json_str = response_text[start:end]
                    else:
                        json_str = response_text
                    
                    data = json.loads(json_str)
                    
                    if 'result' in data and data['result'] and 'article' in data['result']:
                        articles = data['result']['article']
                        print(f"获取到 {len(articles)} 条新闻")
                        
                        # 显示前5条新闻的日期，看排序效果
                        dates = []
                        for i, article in enumerate(articles):
                            date = article.get('date', '')
                            title = article.get('title', '').replace('<em>', '').replace('</em>', '')[:40]
                            print(f"  {i+1}. {date} - {title}...")
                            dates.append(date)
                        
                        # 检查是否按时间排序（最新在前）
                        if len(dates) > 1:
                            is_desc_sorted = all(dates[i] >= dates[i+1] for i in range(len(dates)-1) if dates[i] and dates[i+1])
                            print(f"是否按时间倒序排列: {is_desc_sorted}")
                    else:
                        print("没有获取到新闻")
                        
            except Exception as e:
                print(f"测试失败: {e}")
            
            time.sleep(1)
        """测试不同的排序参数"""
        
        sort_options = [
            "",              # 空字符串（默认）
            "default",       # 默认
            "time",          # 时间
            "relate",        # 相关度
            "score",         # 分数
            "1",             # 数字值
            "2", 
            "3",
            "createTime",    # 创建时间
            "publishTime",   # 发布时间
            "desc",          # 降序
            "asc",           # 升序
        ]
        
        for sort_param in sort_options:
            print(f"\n=== 测试排序参数: '{sort_param}' ===")
            
            param_data = {
                "uid": "",
                "keyword": keyword,
                "type": ["article"],
                "client": "web",
                "clientType": "web",
                "clientVersion": "curr",
                "param": {
                    "article": {
                        "searchScope": "",
                        "sort": sort_param,
                        "pageIndex": 1,
                        "pageSize": 5,
                        "preTag": "",
                        "postTag": ""
                    }
                }
            }
            
            params = {
                'cb': f'jQuery{int(time.time() * 1000)}_{int(time.time() * 1000) + 1}',
                'param': json.dumps(param_data)
            }
            
            try:
                response = requests.get(self.api_url, headers=self.headers, params=params, timeout=10)
                if response.status_code == 200:
                    response_text = response.text
                    
                    if response_text.startswith('jQuery'):
                        start = response_text.find('(') + 1
                        end = response_text.rfind(')')
                        json_str = response_text[start:end]
                    else:
                        json_str = response_text
                    
                    data = json.loads(json_str)
                    
                    if 'result' in data and data['result'] and 'article' in data['result']:
                        articles = data['result']['article']
                        print(f"获取到 {len(articles)} 条新闻")
                        
                        # 显示前3条新闻的日期
                        for i, article in enumerate(articles[:3]):
                            date = article.get('date', '')
                            title = article.get('title', '')[:50]
                            print(f"  {i+1}. {date} - {title}...")
                    else:
                        print("没有获取到新闻")
                        
            except Exception as e:
                print(f"测试失败: {e}")
            
            time.sleep(1)
        """
        获取新闻内容 - 简化版，先测试基本功能
        """
        
        print(f"搜索关键词: {keyword}")
        print(f"获取页数: {max_pages}")
        
        all_news = []
        
        for page in range(1, max_pages + 1):
            # 获取新闻列表
            news_list = self._get_news_list(keyword, page)
            if not news_list:
                break
                
            # 获取每页前2条新闻的内容进行测试
            for i, news in enumerate(news_list[:2]):
                news_date = self._parse_date(news.get('raw_date', ''))
                
                print(f"\n正在处理第{i+1}条新闻:")
                print(f"  标题: {news['title'][:60]}...")
                print(f"  日期: {news_date}")
                print(f"  来源: {news['source']}")
                print(f"  URL: {news['url']}")
                
                content = self._fetch_news_content(news['url'])
                if content:
                    all_news.append({
                        'title': news['title'].replace('<em>', '').replace('</em>', ''),  # 清理高亮标签
                        'date': news_date,
                        'url': news['url'],
                        'source': news['source'],
                        'content': content
                    })
                    print(f"  ✓ 内容获取成功 ({len(content)} 字符)")
                else:
                    print(f"  ✗ 内容获取失败")
                    
                time.sleep(1)
            
            time.sleep(2)
        
        # 转换为DataFrame并按时间排序
        if all_news:
            df = pd.DataFrame(all_news)
            try:
                df['date'] = pd.to_datetime(df['date'])
                df = df.sort_values('date', ascending=False).reset_index(drop=True)
            except:
                pass  # 如果日期解析失败，不排序
            
            print(f"\n总共获取 {len(df)} 条新闻内容")
            return df
        else:
            print("未获取到任何新闻内容")
            return pd.DataFrame()
    
    def _get_news_list(self, keyword, page_index):
        """获取新闻列表"""
        param_data = {
            "uid": "",
            "keyword": keyword,
            "type": ["article"],
            "client": "web",
            "clientType": "web",
            "clientVersion": "curr",
            "param": {
                "article": {
                    "searchScope": "",
                    "sort": "time",  # 使用正确的时间排序参数
                    "pageIndex": page_index,
                    "pageSize": 20,
                    "preTag": "",
                    "postTag": ""
                }
            }
        }
        
        params = {
            'cb': f'jQuery{int(time.time() * 1000)}_{int(time.time() * 1000) + 1}',
            'param': json.dumps(param_data)
        }
        
        try:
            response = requests.get(self.api_url, headers=self.headers, params=params, timeout=15)
            if response.status_code == 200:
                response_text = response.text
                
                if response_text.startswith('jQuery'):
                    start = response_text.find('(') + 1
                    end = response_text.rfind(')')
                    json_str = response_text[start:end]
                else:
                    json_str = response_text
                
                data = json.loads(json_str)
                
                if 'result' in data and data['result'] and 'article' in data['result']:
                    articles = data['result']['article']
                    print(f"第{page_index}页获取到 {len(articles)} 条新闻")
                    
                    # 调试第一条新闻的字段
                    if articles:
                        first_article = articles[0]
                        print(f"新闻字段: {list(first_article.keys())}")
                        print(f"date: {first_article.get('date')}")
                        print(f"示例标题: {first_article.get('title')[:50]}...")
                    
                    news_list = []
                    for article in articles:
                        # 使用正确的日期字段
                        article_date = article.get('date', '')
                        formatted_date = self._parse_date(article_date)
                        
                        news_list.append({
                            'title': article.get('title', '').strip(),
                            'url': article.get('url', ''),
                            'date': formatted_date,
                            'source': article.get('nickname', ''),  # 使用nickname作为来源
                            'raw_date': article_date  # 保留原始日期用于调试
                        })
                    
                    return news_list
                    
        except Exception as e:
            print(f"获取第{page_index}页失败: {e}")
        
        return []
    
    def _fetch_news_content(self, url):
        """获取新闻正文内容"""
        if not url:
            return ""
            
        try:
            response = requests.get(url, headers=self.headers, timeout=10)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                
                # 尝试多种可能的正文选择器
                content_selectors = [
                    '.ArticleBody',
                    '.content',
                    '.article-content',
                    '#ContentBody',
                    '.main-content',
                    'article',
                    '.post-content'
                ]
                
                for selector in content_selectors:
                    content_div = soup.select_one(selector)
                    if content_div:
                        text = content_div.get_text().strip()
                        text = re.sub(r'\s+', ' ', text)
                        if len(text) > 100:
                            return text
                
                # 如果没找到，尝试获取所有p标签内容
                paragraphs = soup.find_all('p')
                if paragraphs:
                    content = ' '.join([p.get_text().strip() for p in paragraphs])
                    content = re.sub(r'\s+', ' ', content)
                    if len(content) > 100:
                        return content
                        
        except Exception as e:
            print(f"获取内容失败 {url}: {e}")
        
        return ""
    
    def _parse_date(self, date_str):
        """解析日期字符串"""
        if not date_str:
            return ""
        
        try:
            # 处理ISO格式
            if 'T' in str(date_str):
                dt = datetime.fromisoformat(str(date_str).replace('Z', ''))
                return dt.strftime('%Y-%m-%d %H:%M:%S')
            
            # 处理其他常见格式
            date_str = str(date_str).strip()
            
            if re.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', date_str):
                return date_str
            
            if re.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}', date_str):
                return date_str + ':00'
            
            if re.match(r'\d{4}-\d{2}-\d{2}', date_str):
                return date_str + ' 00:00:00'
                
        except Exception:
            pass
        
        return date_str
    
    def _is_in_date_range(self, news_date, start_date, end_date):
        """检查日期是否在指定范围内"""
        try:
            if not news_date:
                return True
                
            news_dt = datetime.strptime(news_date[:10], '%Y-%m-%d')
            start_dt = datetime.strptime(start_date, '%Y-%m-%d')
            end_dt = datetime.strptime(end_date, '%Y-%m-%d')
            
            return start_dt <= news_dt <= end_dt
        except Exception:
            return True

# 使用示例
if __name__ == "__main__":
    crawler = EastmoneyNewsCrawler()
    
    # 获取紫金矿业最近一周的新闻内容
    df = crawler.get_news_content("紫金矿业", max_pages=2)
    
    if not df.empty:
        print(f"\n获取到 {len(df)} 条新闻")
        print("\n最新3条新闻:")
        for i, row in df.head(3).iterrows():
            print(f"\n{i+1}. {row['title']}")
            print(f"时间: {row['date']}")
            print(f"来源: {row['source']}")
            print(f"内容: {row['content'][:200]}...")
        
        # 保存到CSV
        df.to_csv('news_content.csv', index=False, encoding='utf-8-sig')
        print(f"\n数据已保存到 news_content.csv")
    else:
        print("未获取到新闻数据")

搜索关键词: 紫金矿业
最大获取 10 条新闻
第1页获取到 20 条新闻
新闻字段: ['id', 'code', 'title', 'postId', 'nickname', 'content', 'date', 'articleType', 'portrait', 'listImage', 'url', 'authorUrl', 'uid', 'likeNum', 'commentNum']
date: 2023-12-28 08:30:00
示例标题: 后浪森林·商业史 | <em>紫金矿业</em>三十二年，人人都是大富翁（四）...

正在处理: 后浪森林·商业史 | <em>紫金矿业</em>三十二年，人人都是大富翁（四）...
  日期: 2023-12-28 08:30:00
  ✓ 成功获取内容 (4089 字符)

正在处理: <em>紫金矿业</em>海外扩张频频触雷...
  日期: 2023-12-15 09:05:39
  ✓ 成功获取内容 (1227 字符)

正在处理: 财大气粗的<em>紫金矿业</em>差钱了...
  日期: 2024-06-21 18:33:52
  ✓ 成功获取内容 (1078 字符)

正在处理: <em>紫金矿业</em>，爆了！...
  日期: 2025-02-13 16:28:05
  ✓ 成功获取内容 (1527 字符)

正在处理: <em>紫金矿业</em>：拟收购藏格矿业股份 可能涉及控制权变更...
  日期: 2025-01-11 09:00:00
  ✓ 成功获取内容 (504 字符)

正在处理: <em>紫金矿业</em>，海外金矿被非法开采...
  日期: 2024-11-21 17:03:02
  ✓ 成功获取内容 (2167 字符)

正在处理: 后浪森林·商业史 | <em>紫金矿业</em>并购史，及其反周期逻辑（三）...
  日期: 2023-12-27 08:30:00
  ✓ 成功获取内容 (2565 字符)

正在处理: <em>紫金矿业</em>，彻底爆了！...
  日期: 2025-02-06 17:11:03
  ✓ 成功获取内容 (1800 字符)

正在处理: 热衷买矿的<em>紫金矿业</em>，又大幅融资补血了...
  日期: 2024-0

In [27]:
import requests
import json
import pandas as pd
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import re

class EastmoneyNewsCrawler:
    def __init__(self):
        # 尝试old_search API
        self.api_url = "https://searchapi.eastmoney.com/bussiness/Web/GetSearchList"
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Referer': 'https://so.eastmoney.com/',
            'Accept': 'application/json, text/javascript, */*; q=0.01',
        }
    
    def test_old_search_api(self, keyword):
        """测试old_search API"""
        
        # 根据之前看到的searchReport等函数，尝试这种参数格式
        params = {
            'cb': f'jQuery{int(time.time() * 1000)}_{int(time.time() * 1000) + 1}',
            'keyword': keyword,
            'type': 1,  # 新闻类型
            'pageindex': 1,
            'pagesize': 10,
            'name': 'normal'
        }
        
        print(f"测试 old_search API: {self.api_url}")
        print(f"参数: {params}")
        
        try:
            response = requests.get(self.api_url, headers=self.headers, params=params, timeout=15)
            print(f"状态码: {response.status_code}")
            print(f"响应长度: {len(response.text)}")
            print(f"响应前500字符: {response.text[:500]}")
            
            if response.status_code == 200:
                response_text = response.text
                
                # 处理JSONP
                if response_text.startswith('jQuery'):
                    start = response_text.find('(') + 1
                    end = response_text.rfind(')')
                    json_str = response_text[start:end]
                else:
                    json_str = response_text
                
                try:
                    data = json.loads(json_str)
                    print(f"JSON解析成功")
                    print(f"数据结构: {type(data)}")
                    if isinstance(data, dict):
                        print(f"顶层字段: {list(data.keys())}")
                        
                        # 查找新闻数据
                        for key, value in data.items():
                            if isinstance(value, list) and value:
                                print(f"找到列表字段 '{key}': {len(value)} 个项目")
                                if value:
                                    print(f"第一个项目字段: {list(value[0].keys()) if isinstance(value[0], dict) else type(value[0])}")
                                    
                except json.JSONDecodeError as e:
                    print(f"JSON解析失败: {e}")
                    
        except Exception as e:
            print(f"请求失败: {e}")

    def test_browser_params(self, keyword):
        """测试浏览器观察到的确切参数"""
        
        # 完全按照你看到的参数
        param_data = {
            "uid": "",
            "keyword": keyword,
            "type": ["cmsArticleWebOld"],
            "client": "web",
            "clientType": "web",
            "clientVersion": "curr",
            "param": {
                "cmsArticleWebOld": {
                    "searchScope": "default",
                    "sort": "time",
                    "pageIndex": 1,
                    "pageSize": 10,
                    "preTag": "<em>",
                    "postTag": "</em>"
                }
            }
        }
        
        params = {
            'cb': f'jQuery{int(time.time() * 1000)}_{int(time.time() * 1000) + 1}',
            'param': json.dumps(param_data)
        }
        
        print("测试浏览器观察到的参数:")
        print(f"API URL: {self.api_url}")
        print(f"参数: {json.dumps(param_data, ensure_ascii=False, indent=2)}")
        
        try:
            response = requests.get(self.api_url, headers=self.headers, params=params, timeout=15)
            print(f"状态码: {response.status_code}")
            print(f"响应长度: {len(response.text)}")
            
            if response.status_code == 200:
                response_text = response.text
                print(f"响应前300字符: {response_text[:300]}")
                
                if response_text.startswith('jQuery'):
                    start = response_text.find('(') + 1
                    end = response_text.rfind(')')
                    json_str = response_text[start:end]
                else:
                    json_str = response_text
                
                try:
                    data = json.loads(json_str)
                    print(f"JSON解析成功")
                    print(f"顶层字段: {list(data.keys())}")
                    
                    if 'result' in data:
                        result = data['result']
                        print(f"result字段: {list(result.keys()) if isinstance(result, dict) else type(result)}")
                        
                        if 'cmsArticleWebOld' in result:
                            articles = result['cmsArticleWebOld']
                            print(f"找到 {len(articles)} 条新闻")
                            
                            if articles:
                                print("前5条新闻:")
                                for i, article in enumerate(articles[:5]):
                                    print(f"  {i+1}. 字段: {list(article.keys())}")
                                    print(f"     标题: {article.get('title', '')[:50]}...")
                                    print(f"     showTime: {article.get('showTime')}")
                                    print(f"     date: {article.get('date')}")
                                    print()
                        else:
                            print("result中没有cmsArticleWebOld字段")
                    else:
                        print("响应中没有result字段")
                        
                except json.JSONDecodeError as e:
                    print(f"JSON解析失败: {e}")
                    
        except Exception as e:
            print(f"请求失败: {e}")

    def get_news_content(self, keyword, max_pages=2, max_news=10):
        """
        获取新闻内容（简化版）
        
        Args:
            keyword: 搜索关键词
            max_pages: 最大页数
            max_news: 最大新闻数量
            
        Returns:
            DataFrame: 包含新闻标题、时间、内容的数据框，按时间排序
        """
        
        print(f"搜索关键词: {keyword}")
        print(f"最大获取 {max_news} 条新闻")
        
        all_news = []
        
        for page in range(1, max_pages + 1):
            if len(all_news) >= max_news:
                break
                
            # 获取新闻列表
            news_list = self._get_news_list(keyword, page)
            if not news_list:
                break
                
            # 处理新闻
            for news in news_list:
                if len(all_news) >= max_news:
                    break
                    
                news_date = self._parse_date(news.get('raw_date', ''))
                
                print(f"\n正在处理: {news['title'][:50]}...")
                print(f"  日期: {news_date}")
                
                content = self._fetch_news_content(news['url'])
                if content and len(content) > 100:  # 确保内容有意义
                    all_news.append({
                        'title': news['title'].replace('<em>', '').replace('</em>', ''),
                        'date': news_date,
                        'url': news['url'],
                        'source': news['source'],
                        'content': content
                    })
                    print(f"  ✓ 成功获取内容 ({len(content)} 字符)")
                else:
                    print(f"  ✗ 内容获取失败或内容太短")
                    
                time.sleep(1)
            
            time.sleep(2)
        
        # 转换为DataFrame并手动按时间排序
        if all_news:
            df = pd.DataFrame(all_news)
            
            # 手动按时间排序（最新在前）
            try:
                df['date_parsed'] = pd.to_datetime(df['date'], errors='coerce')
                df = df.sort_values('date_parsed', ascending=False, na_last=True)
                df = df.drop('date_parsed', axis=1).reset_index(drop=True)
                print(f"\n成功获取 {len(df)} 条新闻，已按时间排序")
            except Exception as e:
                print(f"时间排序失败: {e}")
            
            return df
        else:
            print("未获取到任何新闻内容")
            return pd.DataFrame()
        """
        获取新闻内容 - 简化版，先测试基本功能
        """
        
        print(f"搜索关键词: {keyword}")
        print(f"获取页数: {max_pages}")
        
        all_news = []
        
        for page in range(1, max_pages + 1):
            # 获取新闻列表
            news_list = self._get_news_list(keyword, page)
            if not news_list:
                break
                
            # 获取每页前2条新闻的内容进行测试
            for i, news in enumerate(news_list[:2]):
                news_date = self._parse_date(news.get('raw_date', ''))
                
                print(f"\n正在处理第{i+1}条新闻:")
                print(f"  标题: {news['title'][:60]}...")
                print(f"  日期: {news_date}")
                print(f"  来源: {news['source']}")
                print(f"  URL: {news['url']}")
                
                content = self._fetch_news_content(news['url'])
                if content:
                    all_news.append({
                        'title': news['title'].replace('<em>', '').replace('</em>', ''),  # 清理高亮标签
                        'date': news_date,
                        'url': news['url'],
                        'source': news['source'],
                        'content': content
                    })
                    print(f"  ✓ 内容获取成功 ({len(content)} 字符)")
                else:
                    print(f"  ✗ 内容获取失败")
                    
                time.sleep(1)
            
            time.sleep(2)
        
        # 转换为DataFrame并按时间排序
        if all_news:
            df = pd.DataFrame(all_news)
            try:
                df['date'] = pd.to_datetime(df['date'])
                df = df.sort_values('date', ascending=False).reset_index(drop=True)
            except:
                pass  # 如果日期解析失败，不排序
            
            print(f"\n总共获取 {len(df)} 条新闻内容")
            return df
        else:
            print("未获取到任何新闻内容")
            return pd.DataFrame()

    def test_sorting(self, keyword):
        """测试不同排序参数的效果"""
        
        sort_values = ["default", "relate", "time", "", "1", "2"]
        
        for sort_val in sort_values:
            print(f"\n=== 测试排序参数: '{sort_val}' ===")
            
            param_data = {
                "uid": "",
                "keyword": keyword,
                "type": ["article"],
                "client": "web",
                "clientType": "web",
                "clientVersion": "curr",
                "param": {
                    "article": {
                        "searchScope": "",
                        "sort": sort_val,
                        "pageIndex": 1,
                        "pageSize": 5,
                        "preTag": "",
                        "postTag": ""
                    }
                }
            }
            
            params = {
                'cb': f'jQuery{int(time.time() * 1000)}_{int(time.time() * 1000) + 1}',
                'param': json.dumps(param_data)
            }
            
            try:
                response = requests.get(self.api_url, headers=self.headers, params=params, timeout=10)
                if response.status_code == 200:
                    response_text = response.text
                    
                    if response_text.startswith('jQuery'):
                        start = response_text.find('(') + 1
                        end = response_text.rfind(')')
                        json_str = response_text[start:end]
                    else:
                        json_str = response_text
                    
                    data = json.loads(json_str)
                    
                    if 'result' in data and data['result'] and 'article' in data['result']:
                        articles = data['result']['article']
                        print(f"获取到 {len(articles)} 条新闻")
                        
                        # 显示前5条新闻的日期，看排序效果
                        dates = []
                        for i, article in enumerate(articles):
                            date = article.get('date', '')
                            title = article.get('title', '').replace('<em>', '').replace('</em>', '')[:40]
                            print(f"  {i+1}. {date} - {title}...")
                            dates.append(date)
                        
                        # 检查是否按时间排序（最新在前）
                        if len(dates) > 1:
                            is_desc_sorted = all(dates[i] >= dates[i+1] for i in range(len(dates)-1) if dates[i] and dates[i+1])
                            print(f"是否按时间倒序排列: {is_desc_sorted}")
                    else:
                        print("没有获取到新闻")
                        
            except Exception as e:
                print(f"测试失败: {e}")
            
            time.sleep(1)
        """测试不同的排序参数"""
        
        sort_options = [
            "",              # 空字符串（默认）
            "default",       # 默认
            "time",          # 时间
            "relate",        # 相关度
            "score",         # 分数
            "1",             # 数字值
            "2", 
            "3",
            "createTime",    # 创建时间
            "publishTime",   # 发布时间
            "desc",          # 降序
            "asc",           # 升序
        ]
        
        for sort_param in sort_options:
            print(f"\n=== 测试排序参数: '{sort_param}' ===")
            
            param_data = {
                "uid": "",
                "keyword": keyword,
                "type": ["article"],
                "client": "web",
                "clientType": "web",
                "clientVersion": "curr",
                "param": {
                    "article": {
                        "searchScope": "",
                        "sort": sort_param,
                        "pageIndex": 1,
                        "pageSize": 5,
                        "preTag": "",
                        "postTag": ""
                    }
                }
            }
            
            params = {
                'cb': f'jQuery{int(time.time() * 1000)}_{int(time.time() * 1000) + 1}',
                'param': json.dumps(param_data)
            }
            
            try:
                response = requests.get(self.api_url, headers=self.headers, params=params, timeout=10)
                if response.status_code == 200:
                    response_text = response.text
                    
                    if response_text.startswith('jQuery'):
                        start = response_text.find('(') + 1
                        end = response_text.rfind(')')
                        json_str = response_text[start:end]
                    else:
                        json_str = response_text
                    
                    data = json.loads(json_str)
                    
                    if 'result' in data and data['result'] and 'article' in data['result']:
                        articles = data['result']['article']
                        print(f"获取到 {len(articles)} 条新闻")
                        
                        # 显示前3条新闻的日期
                        for i, article in enumerate(articles[:3]):
                            date = article.get('date', '')
                            title = article.get('title', '')[:50]
                            print(f"  {i+1}. {date} - {title}...")
                    else:
                        print("没有获取到新闻")
                        
            except Exception as e:
                print(f"测试失败: {e}")
            
            time.sleep(1)
        """
        获取新闻内容 - 简化版，先测试基本功能
        """
        
        print(f"搜索关键词: {keyword}")
        print(f"获取页数: {max_pages}")
        
        all_news = []
        
        for page in range(1, max_pages + 1):
            # 获取新闻列表
            news_list = self._get_news_list(keyword, page)
            if not news_list:
                break
                
            # 获取每页前2条新闻的内容进行测试
            for i, news in enumerate(news_list[:2]):
                news_date = self._parse_date(news.get('raw_date', ''))
                
                print(f"\n正在处理第{i+1}条新闻:")
                print(f"  标题: {news['title'][:60]}...")
                print(f"  日期: {news_date}")
                print(f"  来源: {news['source']}")
                print(f"  URL: {news['url']}")
                
                content = self._fetch_news_content(news['url'])
                if content:
                    all_news.append({
                        'title': news['title'].replace('<em>', '').replace('</em>', ''),  # 清理高亮标签
                        'date': news_date,
                        'url': news['url'],
                        'source': news['source'],
                        'content': content
                    })
                    print(f"  ✓ 内容获取成功 ({len(content)} 字符)")
                else:
                    print(f"  ✗ 内容获取失败")
                    
                time.sleep(1)
            
            time.sleep(2)
        
        # 转换为DataFrame并按时间排序
        if all_news:
            df = pd.DataFrame(all_news)
            try:
                df['date'] = pd.to_datetime(df['date'])
                df = df.sort_values('date', ascending=False).reset_index(drop=True)
            except:
                pass  # 如果日期解析失败，不排序
            
            print(f"\n总共获取 {len(df)} 条新闻内容")
            return df
        else:
            print("未获取到任何新闻内容")
            return pd.DataFrame()
    
    def _get_news_list(self, keyword, page_index):
        """获取新闻列表"""
        param_data = {
            "uid": "",
            "keyword": keyword,
            "type": ["article"],
            "client": "web",
            "clientType": "web",
            "clientVersion": "curr",
            "param": {
                "article": {
                    "searchScope": "",
                    "sort": "time",  # 使用正确的时间排序参数
                    "pageIndex": page_index,
                    "pageSize": 20,
                    "preTag": "",
                    "postTag": ""
                }
            }
        }
        
        params = {
            'cb': f'jQuery{int(time.time() * 1000)}_{int(time.time() * 1000) + 1}',
            'param': json.dumps(param_data)
        }
        
        try:
            response = requests.get(self.api_url, headers=self.headers, params=params, timeout=15)
            if response.status_code == 200:
                response_text = response.text
                
                if response_text.startswith('jQuery'):
                    start = response_text.find('(') + 1
                    end = response_text.rfind(')')
                    json_str = response_text[start:end]
                else:
                    json_str = response_text
                
                data = json.loads(json_str)
                
                if 'result' in data and data['result'] and 'article' in data['result']:
                    articles = data['result']['article']
                    print(f"第{page_index}页获取到 {len(articles)} 条新闻")
                    
                    # 调试第一条新闻的字段
                    if articles:
                        first_article = articles[0]
                        print(f"新闻字段: {list(first_article.keys())}")
                        print(f"date: {first_article.get('date')}")
                        print(f"示例标题: {first_article.get('title')[:50]}...")
                    
                    news_list = []
                    for article in articles:
                        # 使用正确的日期字段
                        article_date = article.get('date', '')
                        formatted_date = self._parse_date(article_date)
                        
                        news_list.append({
                            'title': article.get('title', '').strip(),
                            'url': article.get('url', ''),
                            'date': formatted_date,
                            'source': article.get('nickname', ''),  # 使用nickname作为来源
                            'raw_date': article_date  # 保留原始日期用于调试
                        })
                    
                    return news_list
                    
        except Exception as e:
            print(f"获取第{page_index}页失败: {e}")
        
        return []
    
    def _fetch_news_content(self, url):
        """获取新闻正文内容"""
        if not url:
            return ""
            
        try:
            response = requests.get(url, headers=self.headers, timeout=10)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                
                # 尝试多种可能的正文选择器
                content_selectors = [
                    '.ArticleBody',
                    '.content',
                    '.article-content',
                    '#ContentBody',
                    '.main-content',
                    'article',
                    '.post-content'
                ]
                
                for selector in content_selectors:
                    content_div = soup.select_one(selector)
                    if content_div:
                        text = content_div.get_text().strip()
                        text = re.sub(r'\s+', ' ', text)
                        if len(text) > 100:
                            return text
                
                # 如果没找到，尝试获取所有p标签内容
                paragraphs = soup.find_all('p')
                if paragraphs:
                    content = ' '.join([p.get_text().strip() for p in paragraphs])
                    content = re.sub(r'\s+', ' ', content)
                    if len(content) > 100:
                        return content
                        
        except Exception as e:
            print(f"获取内容失败 {url}: {e}")
        
        return ""
    
    def _parse_date(self, date_str):
        """解析日期字符串"""
        if not date_str:
            return ""
        
        try:
            # 处理ISO格式
            if 'T' in str(date_str):
                dt = datetime.fromisoformat(str(date_str).replace('Z', ''))
                return dt.strftime('%Y-%m-%d %H:%M:%S')
            
            # 处理其他常见格式
            date_str = str(date_str).strip()
            
            if re.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', date_str):
                return date_str
            
            if re.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}', date_str):
                return date_str + ':00'
            
            if re.match(r'\d{4}-\d{2}-\d{2}', date_str):
                return date_str + ' 00:00:00'
                
        except Exception:
            pass
        
        return date_str
    
    def _is_in_date_range(self, news_date, start_date, end_date):
        """检查日期是否在指定范围内"""
        try:
            if not news_date:
                return True
                
            news_dt = datetime.strptime(news_date[:10], '%Y-%m-%d')
            start_dt = datetime.strptime(start_date, '%Y-%m-%d')
            end_dt = datetime.strptime(end_date, '%Y-%m-%d')
            
            return start_dt <= news_dt <= end_dt
        except Exception:
            return True

# 使用示例
if __name__ == "__main__":
    crawler = EastmoneyNewsCrawler()
    
    # 获取紫金矿业最近一周的新闻内容
    df = crawler.get_news_content("紫金矿业", max_pages=2)
    
    if not df.empty:
        print(f"\n获取到 {len(df)} 条新闻")
        print("\n最新3条新闻:")
        for i, row in df.head(3).iterrows():
            print(f"\n{i+1}. {row['title']}")
            print(f"时间: {row['date']}")
            print(f"来源: {row['source']}")
            print(f"内容: {row['content'][:200]}...")
        
        # 保存到CSV
        df.to_csv('news_content.csv', index=False, encoding='utf-8-sig')
        print(f"\n数据已保存到 news_content.csv")
    else:
        print("未获取到新闻数据")

搜索关键词: 紫金矿业
最大获取 10 条新闻
未获取到任何新闻内容
未获取到新闻数据


In [28]:
import requests
import pandas as pd
import time
from datetime import datetime
from bs4 import BeautifulSoup
import re

class EastmoneyNewsCrawler:
    def __init__(self):
        # 回到最初能工作的API
        self.api_url = "https://search-api-web.eastmoney.com/search/jsonp"
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Referer': 'https://so.eastmoney.com/',
            'Accept': 'application/json, text/javascript, */*; q=0.01',
        }
    
    def get_news_simple(self, keyword, max_news=10):
        """
        简化版：只获取新闻列表和内容，不做复杂的时间排序
        """
        print(f"搜索关键词: {keyword}")
        
        # 使用最初能工作的参数
        param_data = {
            "uid": "",
            "keyword": keyword,
            "type": ["article"],
            "client": "web",
            "clientType": "web",
            "clientVersion": "curr",
            "param": {
                "article": {
                    "searchScope": "",
                    "sort": "",
                    "pageIndex": 1,
                    "pageSize": max_news,
                    "preTag": "",
                    "postTag": ""
                }
            }
        }
        
        params = {
            'cb': f'jQuery{int(time.time() * 1000)}_{int(time.time() * 1000) + 1}',
            'param': json.dumps(param_data)
        }
        
        try:
            response = requests.get(self.api_url, headers=self.headers, params=params, timeout=15)
            
            if response.status_code == 200:
                response_text = response.text
                
                if response_text.startswith('jQuery'):
                    start = response_text.find('(') + 1
                    end = response_text.rfind(')')
                    json_str = response_text[start:end]
                else:
                    json_str = response_text
                
                data = json.loads(json_str)
                
                if 'result' in data and data['result'] and 'article' in data['result']:
                    articles = data['result']['article']
                    print(f"获取到 {len(articles)} 条新闻")
                    
                    news_data = []
                    for i, article in enumerate(articles[:max_news]):
                        print(f"\n处理第{i+1}条新闻:")
                        print(f"标题: {article.get('title', '')[:50]}...")
                        print(f"日期: {article.get('date', '')}")
                        print(f"URL: {article.get('url', '')}")
                        
                        # 获取新闻内容
                        content = self._get_content(article.get('url', ''))
                        
                        news_item = {
                            'title': self._clean_title(article.get('title', '')),
                            'date': article.get('date', ''),
                            'url': article.get('url', ''),
                            'source': article.get('nickname', ''),
                            'content': content
                        }
                        
                        if content:
                            news_data.append(news_item)
                            print(f"✓ 内容获取成功 ({len(content)} 字符)")
                        else:
                            print("✗ 内容获取失败")
                        
                        time.sleep(1)
                    
                    if news_data:
                        df = pd.DataFrame(news_data)
                        print(f"\n成功获取 {len(df)} 条完整新闻")
                        return df
                    
        except Exception as e:
            print(f"获取失败: {e}")
        
        return pd.DataFrame()
    
    def _clean_title(self, title):
        """清理标题"""
        return title.replace('<em>', '').replace('</em>', '').strip()
    
    def _get_content(self, url):
        """获取新闻正文"""
        if not url:
            return ""
        
        try:
            response = requests.get(url, headers=self.headers, timeout=10)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                
                # 尝试多种内容选择器
                selectors = [
                    '.ArticleBody',
                    '.content', 
                    '.article-content',
                    '#ContentBody',
                    'article',
                    '.post-content'
                ]
                
                for selector in selectors:
                    content_div = soup.select_one(selector)
                    if content_div:
                        text = content_div.get_text().strip()
                        text = re.sub(r'\s+', ' ', text)
                        if len(text) > 100:
                            return text
                
                # 备用方案：获取所有p标签
                paragraphs = soup.find_all('p')
                if paragraphs:
                    content = ' '.join([p.get_text().strip() for p in paragraphs])
                    content = re.sub(r'\s+', ' ', content)
                    if len(content) > 100:
                        return content
                        
        except Exception as e:
            print(f"获取内容失败 {url}: {e}")
        
        return ""

# 使用示例
if __name__ == "__main__":
    crawler = EastmoneyNewsCrawler()
    
    # 获取新闻
    df = crawler.get_news_simple("紫金矿业", max_news=5)
    
    if not df.empty:
        print(f"\n成功获取 {len(df)} 条新闻")
        
        # 显示结果
        for i, row in df.iterrows():
            print(f"\n{i+1}. {row['title']}")
            print(f"日期: {row['date']}")
            print(f"内容: {row['content'][:200]}...")
        
        # 保存
        df.to_csv('news_simple.csv', index=False, encoding='utf-8-sig')
        print("\n数据已保存到 news_simple.csv")
    else:
        print("未获取到新闻")





        

搜索关键词: 紫金矿业
获取到 5 条新闻

处理第1条新闻:
标题: 后浪森林·商业史 | <em>紫金矿业</em>三十二年，人人都是大富翁（四）...
日期: 2023-12-28 08:30:00
URL: http://caifuhao.eastmoney.com/news/20231228004008763123340
✓ 内容获取成功 (4089 字符)

处理第2条新闻:
标题: <em>紫金矿业</em>，爆了！...
日期: 2025-02-13 16:28:05
URL: http://caifuhao.eastmoney.com/news/20250213162805586513200
✓ 内容获取成功 (1527 字符)

处理第3条新闻:
标题: <em>紫金矿业</em>，海外金矿被非法开采...
日期: 2024-11-21 17:03:02
URL: http://caifuhao.eastmoney.com/news/20241121170302175921210
✓ 内容获取成功 (2167 字符)

处理第4条新闻:
标题: 热衷买矿的<em>紫金矿业</em>，又大幅融资补血了...
日期: 2024-06-19 23:41:01
URL: http://caifuhao.eastmoney.com/news/20240619234101211306570
✓ 内容获取成功 (3841 字符)

处理第5条新闻:
标题: 财大气粗的<em>紫金矿业</em>差钱了...
日期: 2024-06-21 18:33:52
URL: http://caifuhao.eastmoney.com/news/20240621183352621803200
✓ 内容获取成功 (1078 字符)

成功获取 5 条完整新闻

成功获取 5 条新闻

1. 后浪森林·商业史 | 紫金矿业三十二年，人人都是大富翁（四）
日期: 2023-12-28 08:30:00
内容: 后浪森林研究室|三文 编辑|罗周 统筹、编辑助理|许佳维 紫金矿业改变了太多人的命运。 从陈景河、陈发树、柯希平，到同康村民、紫金职工，再到基金经理邓晓峰等等。最新数据是紫金矿业持股机构1195家。 紫金矿业的财富游戏，人人都是大富翁。 原始股…… 1996年，紫金矿山投产